In [29]:
!pip install uszipcode

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from uszipcode import Zipcode, SearchEngine

In [31]:
!unzip airbnb.csv.zip

Archive:  airbnb.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [71]:
airbnb = pd.read_csv("airbnb.csv.zip")
print(airbnb.shape)
airbnb.head()

(74111, 29)


,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [72]:
airbnb.isnull().sum()

id                            0
log_price                     0
property_type                 0
room_type                     0
amenities                     0
accommodates                  0
bathrooms                   200
bed_type                      0
cancellation_policy           0
cleaning_fee                  0
city                          0
description                   0
first_review              15864
host_has_profile_pic        188
host_identity_verified      188
host_response_rate        18299
host_since                  188
instant_bookable              0
last_review               15827
latitude                      0
longitude                     0
name                          0
neighbourhood              6872
number_of_reviews             0
review_scores_rating      16722
thumbnail_url              8216
zipcode                     966
bedrooms                     91
beds                        131
dtype: int64

In [73]:
airbnb['host_has_profile_pic'].value_counts()

t    73697
f      226
Name: host_has_profile_pic, dtype: int64

In [74]:
missing_df = airbnb.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
missing_df

,column_name,missing_count
27,bedrooms,91
28,beds,131
13,host_has_profile_pic,188
14,host_identity_verified,188
16,host_since,188
6,bathrooms,200
26,zipcode,966
22,neighbourhood,6872
25,thumbnail_url,8216
18,last_review,15827


In [75]:
search = SearchEngine(simple_zipcode=True)
# 'bathrooms'
airbnb['bathrooms'] = airbnb['bathrooms'].fillna(0)


# 'bedrooms'
airbnb['bedrooms'] = airbnb['bedrooms'].fillna(0)

# 'beds' 
airbnb['beds'] = airbnb['beds'].fillna(0)


# 'host_has_profile_pic'
airbnb.loc[airbnb.host_has_profile_pic == 't', 'host_has_profile_pic'] = 1
airbnb.loc[airbnb.host_has_profile_pic == 'f', 'host_has_profile_pic'] = 0
airbnb['host_has_profile_pic'] = airbnb['host_has_profile_pic'].fillna(0)

# 'neighbourhood'
airbnb['neighbourhood'] = airbnb['neighbourhood'].fillna('Unknown')

# 'review_scores_rating'
airbnb['review_scores_rating'] = airbnb['review_scores_rating'].fillna(0)

# 'host_response_rate'
airbnb['host_response_rate'] = airbnb['host_response_rate'].str.replace('%','')
airbnb['host_response_rate'] = airbnb['host_response_rate'].fillna('0')
airbnb['host_response_rate'] = pd.to_numeric(airbnb['host_response_rate'])

# 'thumbnai_url'
airbnb['thumbnail_url'] = airbnb['thumbnail_url'].fillna('Unknown')

# 'last_review'
airbnb['last_review'] = airbnb['last_review'].fillna('00-00-00')

# 'first_review'
airbnb['first_review'] = airbnb['first_review'].fillna('00-00-00')

# 'host_since'
airbnb['host_since'] = airbnb['host_since'].fillna('00-00-00')

# 'host_identity_verified'
airbnb.loc[airbnb.host_identity_verified == 't', 'host_identity_verified'] = 1
airbnb.loc[airbnb.host_identity_verified == 'f', 'host_identity_verified'] = 0
airbnb['host_identity_verified'] = airbnb['host_identity_verified'].fillna(0)

# 'zipcode'
airbnb['zipcode'] = airbnb['zipcode'].fillna(0)
airbnb.loc[airbnb.zipcode == ' ', 'zipcode'] = 0
idx = airbnb.index[airbnb['zipcode']==0].tolist()
for i in idx:
    lat = airbnb['latitude'][i]
    lon = airbnb['longitude'][i]
    result = np.max(search.by_coordinates(lat, lon, radius=30, returns=5))
    airbnb['zipcode'][i]=result.values()[0]    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [76]:
missing_df = airbnb.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
missing_df

,column_name,missing_count


In [77]:
airbnb['int_price'] = np.exp(airbnb['log_price'])

In [79]:
airbnb.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,int_price
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,1,1,0,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0,150.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,1,0,100,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0,169.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,1,1,100,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0,145.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,00-00-00,1,1,0,2015-04-19,f,00-00-00,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,0.0,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0,750.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,1,1,100,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,Unknown,20009,0.0,1.0,115.0


In [80]:
airbnb = airbnb.drop(['instant_bookable', 'name', 'amenities', 'zipcode', 'thumbnail_url', 'host_since',
                                  'first_review', 'description', 'id'], axis=1)

In [83]:
print(airbnb.shape)
airbnb.isnull().sum()

(74111, 21)


log_price                 0
property_type             0
room_type                 0
accommodates              0
bathrooms                 0
bed_type                  0
cancellation_policy       0
cleaning_fee              0
city                      0
host_has_profile_pic      0
host_identity_verified    0
host_response_rate        0
last_review               0
latitude                  0
longitude                 0
neighbourhood             0
number_of_reviews         0
review_scores_rating      0
bedrooms                  0
beds                      0
int_price                 0
dtype: int64